In [ ]:
%load_ext autoreload
%autoreload 2
import logging
import matplotlib.pyplot as plt

# from folium import LayerControl
from eo_tools_dev.util import show_insar_coh, show_insar_phi, show_sar_int
import geopandas as gpd
from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

In [ ]:
# change to your custom locations
data_dir = "/data/S1"
out_dir = f"/data/res/test-iw-refa"

ids = [
 "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
 "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
]
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"

iw = 1 # subswath
pol = "vv" # polarization ("vv"or "vh")
min_burst = 3
max_burst = 3 # Set to None to process all (warning: memory hungry)

# Pre-process InSAR pair

In [ ]:
from eo_tools.S1 import process_refa

process_refa.preprocess_insar_iw(
    primary_dir,
    secondary_dir,
    out_dir,
    iw=iw,
    pol=pol,
    min_burst=min_burst,
    max_burst=max_burst,
    # dem_upsampling=1.8,
    dem_upsampling=1,
    apply_fast_esd=True,
    dem_buffer_arc_sec=50,
    dem_force_download=True,
    dir_dem="/data/tmp/"
)

In [ ]:
from eo_tools.S1 import process

process.preprocess_insar_iw(
    primary_dir,
    secondary_dir,
    out_dir,
    iw=iw,
    pol=pol,
    min_burst=min_burst,
    max_burst=max_burst,
    dem_upsampling=1.8,
    # dem_upsampling=1,
    apply_fast_esd=True,
    dem_buffer_arc_sec=50,
    dem_force_download=True,
    dir_dem="/data/tmp/"
)

# Apply geocoding using lookup tables

In [ ]:
from eo_tools.S1.process import sar2geo, coherence, amplitude
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_amp = f"{out_dir}/amp.tif"
file_coh = f"{out_dir}/coh.tif"
file_phi_geo = f"{out_dir}/phi_geo.tif"
file_amp_geo = f"{out_dir}/amp_geo.tif"
file_coh_geo = f"{out_dir}/coh_geo.tif"
file_lut = f"{out_dir}/lut.tif"

# computing amplitude and complex coherence  in the radar geometry
coherence(file_prm, file_sec, file_coh, box_size=[3,10], magnitude=False)
amplitude(file_prm, file_amp)

# combined multilooking and geocoding
# interferometric coherence
sar2geo(file_coh, file_lut, file_coh_geo, 1, 4, "bicubic", write_phase=False, magnitude_only=True)

# interferometric phase
sar2geo(file_coh, file_lut, file_phi_geo, 1, 4, "bicubic", write_phase=True)

# amplitude of the primary image
sar2geo(file_amp, file_lut, file_amp_geo, 2, 8, "bicubic", False, True)

# Visualize outputs on a folium map

In [ ]:
show_sar_int(file_amp_geo)

In [ ]:
show_insar_phi(file_phi_geo)

In [ ]:
show_insar_coh(file_coh_geo)

In [ ]:
from eo_tools.S1.core import S1IWSwath

sw = S1IWSwath(secondary_dir, iw=1, pol="vv")

In [ ]:
gdf = sw.gdf_burst_geom[(sw.gdf_burst_geom["burst"] >= 3) & (sw.gdf_burst_geom["burst"] <= 6)]

In [ ]:
geom = gdf[gdf["burst"]==3].geometry

In [ ]:
geom.crs = None

In [ ]:
sw.fetch_dem_burst(3)

In [ ]:
geom = gdf.iloc[0].geometry 
geom2 = gdf.iloc[0].geometry.buffer(1000/3600)
# from shapely import difference
# difference(geom2, geom)
print(box(*geom.bounds))

In [ ]:
box(*gdf.union_all().buffer(30/3600).bounds)

In [ ]:
from shapely import box

from eo_tools.dem import retrieve_dem
file_dem1 = "/tmp/test1.tif"
file_dem2 = "/tmp/test2.tif"

buf_arcsec = 3
buf_pix = buf_arcsec / 3600
retrieve_dem(gdf.iloc[2].geometry, file_dem1, 'nasadem')
retrieve_dem(gdf.iloc[2].geometry.buffer(buf_pix), file_dem2, 'nasadem')

In [ ]:
import rasterio
# fd = '/tmp/test1.tif'
# fd = '/data/tmp/dem-b3-s1a-iw1-slc-vv-20230904t063731-20230904t063756-050174-0609e3-004.tiff'
# with rasterio.open(file_dem1) as ds:
with rasterio.open(file_dem1) as ds:
    arr = ds.read(1)
    print(ds.profile)
with rasterio.open(file_dem2) as ds:
    arr2 = ds.read(1)
    print(ds.profile)


print(arr2.shape[0] - arr.shape[0])
print(arr2.shape[1] - arr.shape[1])

In [ ]:
print(box(1,0,0,1).bounds[:2])

In [ ]:
from rasterio.transform import AffineTransformer
import numpy as np
at = AffineTransformer(ds.transform)
at.rowcol(*(box(1,0,0,1).bounds[::2],box(1,0,0,1).bounds[1::2]))
# at.rowcol()

In [ ]:
box(1,0,0,1).bounds[1::2]